<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [ ]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [ ]:
import folium
import pandas as pd

In [ ]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [ ]:
## Task 1: Mark all launch sites on a map
import pandas as pd
import folium

# Load the dataset
URL1 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv"
data_part_2 = pd.read_csv(URL1)

# Extract the launch site coordinates
launch_sites = data_part_2[['LaunchSite', 'Latitude', 'Longitude']].drop_duplicates()

# Create a map centered around the average coordinates of all launch sites
center_lat = launch_sites['Latitude'].mean()
center_lon = launch_sites['Longitude'].mean()
launch_map = folium.Map(location=[center_lat, center_lon], zoom_start=5)

# Add markers for each launch site
for index, site in launch_sites.iterrows():
    folium.Marker(
        location=[site['Latitude'], site['Longitude']],
        popup=site['LaunchSite'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(launch_map)

# Display the map
launch_map.save('launch_sites_map.html')
launch_map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [ ]:
import pandas as pd
import folium
import requests
from io import StringIO

# Download and read the `spacex_launch_geo.csv`
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)
data = StringIO(response.text)
spacex_df = pd.read_csv(data)

# Print the column names to inspect
print(spacex_df.columns)

# Extract the launch site coordinates
# Update the column names based on the actual columns in the DataFrame
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()

# Create a map centered around the average coordinates of all launch sites
center_lat = launch_sites['Lat'].mean()
center_lon = launch_sites['Long'].mean()
launch_map = folium.Map(location=[center_lat, center_lon], zoom_start=5)

# Add markers for each launch site
for index, site in launch_sites.iterrows():
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=site['Launch Site'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(launch_map)

# Save and display the map
launch_map.save('launch_sites_map.html')
launch_map

Index(['Flight Number', 'Date', 'Time (UTC)', 'Booster Version', 'Launch Site',
       'Payload', 'Payload Mass (kg)', 'Orbit', 'Customer', 'Landing Outcome',
       'class', 'Lat', 'Long'],
      dtype='object')


Now, you can take a look at what are the coordinates for each site.


In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [ ]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [ ]:
import pandas as pd
import folium
import requests
from io import StringIO
from folium.map import Marker
from folium.features import DivIcon

# Download and read the `spacex_launch_geo.csv`
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)
data = StringIO(response.text)
spacex_df = pd.read_csv(data)

# Extract the launch site coordinates
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()

# Create a map centered around the average coordinates of all launch sites
center_lat = launch_sites['Lat'].mean()
center_lon = launch_sites['Long'].mean()
site_map = folium.Map(location=[center_lat, center_lon], zoom_start=5)

# Add markers for each launch site
for index, site in launch_sites.iterrows():
    # Create a blue circle with popup label
    circle = folium.Circle(
        location=[site['Lat'], site['Long']],
        radius=1000,
        color='#d35400',
        fill=True,
    ).add_child(folium.Popup(site['Launch Site']))

    # Create a marker with a custom icon as a text label
    marker = folium.Marker(
        location=[site['Lat'], site['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{site["Launch Site"]}</b></div>'
        )
    )

    # Add the circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Save and display the map
site_map.save('launch_sites_map.html')
site_map

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [ ]:
import pandas as pd
import folium
from folium.map import Marker
from folium.features import DivIcon

# Assuming `launch_sites` DataFrame is already defined and contains columns 'Launch Site', 'Lat', and 'Long'

# Example data for illustration (replace with actual data)
launch_sites = pd.DataFrame({
    'Launch Site': ['Site A', 'Site B', 'Site C'],
    'Lat': [29.548, 28.573, 30.501],
    'Long': [-95.097, -80.648, -81.291]
})

# Initial the map
site_map = folium.Map(location=[launch_sites['Lat'].mean(), launch_sites['Long'].mean()], zoom_start=5)

# Add circles and markers for each launch site
for index, site in launch_sites.iterrows():
    # Create a circle with a popup label
    circle = folium.Circle(
        location=[site['Lat'], site['Long']],
        radius=1000,
        color='#000000',  # Black color
        fill=True
    ).add_child(folium.Popup(site['Launch Site']))

    # Create a marker with a custom icon as a text label
    marker = folium.Marker(
        location=[site['Lat'], site['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{site["Launch Site"]}</b></div>'
        )
    )

    # Add the circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Save the map as HTML (optional)
site_map.save('launch_sites_map.html')

# Display the map
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [ ]:
import pandas as pd
import folium
from folium.map import Marker
from folium.features import DivIcon

# Example data for illustration (replace with actual data loading and filtering)
launch_data = pd.DataFrame({
    'Launch Site': ['Site A', 'Site B', 'Site C', 'Site A', 'Site B'],
    'Outcome': ['Success', 'Failure', 'Success', 'Success', 'Failure'],
    'Lat': [29.548, 28.573, 30.501, 29.548, 28.573],
    'Long': [-95.097, -80.648, -81.291, -95.097, -80.648]
})

# Initial the map (using average coordinates of launch sites for centering)
site_map = folium.Map(location=[launch_data['Lat'].mean(), launch_data['Long'].mean()], zoom_start=5)

# Add circles and markers for each launch site
for index, row in launch_data.iterrows():
    # Determine color based on outcome
    if row['Outcome'] == 'Success':
        color = 'green'
    else:
        color = 'red'

    # Create a circle with a popup label
    circle = folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color=color,
        fill=True
    ).add_child(folium.Popup(f"{row['Launch Site']} - {row['Outcome']}"))

    # Create a marker with a custom icon as a text label
    marker = folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:{color};"><b>{row["Launch Site"]}</b></div>'
        )
    )

    # Add the circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Save the map as HTML (optional)
site_map.save('launch_sites_success_failure_map.html')

# Display the map
site_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [ ]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [ ]:
import folium
from folium.plugins import MarkerCluster

# Initialize the map
site_map = folium.Map(location=[29.548, -95.097], zoom_start=5)

# Example launch data (replace with your actual data)
launch_data = [
    {'Launch Site': 'Site A', 'Lat': 29.548, 'Long': -95.097, 'Class': 1},
    {'Launch Site': 'Site B', 'Lat': 28.573, 'Long': -80.648, 'Class': 0},
    {'Launch Site': 'Site C', 'Lat': 30.501, 'Long': -81.291, 'Class': 1},
    {'Launch Site': 'Site A', 'Lat': 29.548, 'Long': -95.097, 'Class': 1},
    {'Launch Site': 'Site B', 'Lat': 28.573, 'Long': -80.648, 'Class': 0}
]

# Create MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers for each launch record
for record in launch_data:
    # Determine color based on launch outcome
    if record['Class'] == 1:
        color = 'green'
    else:
        color = 'red'

    # Create a marker with popup label
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color=color),
        popup=f"{record['Launch Site']} - {'Success' if record['Class'] == 1 else 'Failure'}"
    )

    # Add marker to MarkerCluster
    marker_cluster.add_child(marker)

# Display the map
site_map

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [ ]:
print(launch_sites.columns)

Index(['Launch Site', 'Lat', 'Long'], dtype='object')


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [ ]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities
import pandas as pd

# Example of launch sites data (replace with your actual data)
launch_sites_data = {
    'LaunchSite': ['CCAFS SLC 40', 'KSC LC 39A', 'VAFB SLC 4E'],
    'Lat': [28.562302, 28.573255, 34.632834],
    'Long': [-80.577356, -80.646895, -120.610746]
}

launch_sites = pd.DataFrame(launch_sites_data)
# Example proximity locations (replace with your actual locations)
proximity_locations = {
    'Location': ['Cape Canaveral', 'Kennedy Space Center Visitor Complex', 'Vandenberg Air Force Base'],
    'Lat': [28.3922, 28.5731, 34.7420],
    'Long': [-80.6077, -80.6464, -120.5724]
}

proximities = pd.DataFrame(proximity_locations)
from geopy.distance import distance

# Function to calculate distance between two coordinates
def calculate_distance(site_lat, site_long, loc_lat, loc_long):
    site_coord = (site_lat, site_long)
    loc_coord = (loc_lat, loc_long)
    return distance(site_coord, loc_coord).kilometers

# Iterate through each launch site and proximity location to calculate distances
for index, launch_site in launch_sites.iterrows():
    print(f"Distances from {launch_site['LaunchSite']}:")
    for index, proximity in proximities.iterrows():
        dist = calculate_distance(launch_site['Lat'], launch_site['Long'], proximity['Lat'], proximity['Long'])
        print(f"To {proximity['Location']}: {dist:.2f} km")
    print()

Distances from CCAFS SLC 40:
To Cape Canaveral: 19.08 km
To Kennedy Space Center Visitor Complex: 6.86 km
To Vandenberg Air Force Base: 3829.32 km

Distances from KSC LC 39A:
To Cape Canaveral: 20.43 km
To Kennedy Space Center Visitor Complex: 0.05 km
To Vandenberg Air Force Base: 3822.53 km

Distances from VAFB SLC 4E:
To Cape Canaveral: 3836.91 km
To Kennedy Space Center Visitor Complex: 3826.32 km
To Vandenberg Air Force Base: 12.61 km



Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [ ]:
import folium
from folium.plugins import MousePosition

# Example map initialization
site_map = folium.Map(location=[28.562302, -80.577356], zoom_start=10)

# Add MousePosition control to the map
MousePosition().add_to(site_map)

# Display the map
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of Earth in km
    R = 6373.0

    # Convert latitude and longitude from degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Calculate the differences in longitude and latitude
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Apply the Haversine formula to calculate the distance
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

In [ ]:
import folium
from folium.plugins import MousePosition

# Example: Zoomed in to a specific launch site (Kennedy Space Center)
site_map = folium.Map(location=[28.562302, -80.577356], zoom_start=14)

# Add MousePosition control to the map
MousePosition().add_to(site_map)

# Display the map
site_map

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Example coordinates of the launch site (Kennedy Space Center)
launch_site_lat, launch_site_lon = 28.562302, -80.577356

# Example coordinates of the closest coastline point
coastline_lat, coastline_lon = 28.56367, -80.57163

# Calculate the distance between the launch site and the coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

print(f"Distance to the closest coastline: {distance_coastline:.2f} km")

Distance to the closest coastline: 0.58 km


In [ ]:
# Example coordinates of the launch site (Kennedy Space Center)
launch_site_lat, launch_site_lon = 28.562302, -80.577356

# Example coordinates of the closest coastline point
coastline_lat, coastline_lon = 28.56367, -80.57163

# Calculate the distance between the launch site and the coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a Folium map centered around the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Create a Folium Marker for the closest coastline point
coastline_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=folium.DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>',
    )
)

# Add the coastline marker to the map
site_map.add_child(coastline_marker)

# Display the map
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
import folium

# Example coordinates of the launch site (Kennedy Space Center)
launch_site_lat, launch_site_lon = 28.562302, -80.577356

# Example coordinates of the closest coastline point
coastline_lat, coastline_lon = 28.56367, -80.57163

# Create a list of coordinates for the PolyLine
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]

# Create a PolyLine object
lines = folium.PolyLine(locations=coordinates, weight=1)

# Create a Folium map centered around the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add the PolyLine object to the map
site_map.add_child(lines)

# Display the map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [ ]:
import folium
from math import sin, cos, sqrt, atan2, radians

# Function to calculate distance between two points given their latitude and longitude
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Example coordinates of the launch site (Kennedy Space Center)
launch_site_lat, launch_site_lon = 28.562302, -80.577356

# Example coordinates of the closest city, railway, highway, etc.
poi_lat, poi_lon = 28.533, -81.348

# Calculate the distance between launch site and the point of interest
distance_to_poi = calculate_distance(launch_site_lat, launch_site_lon, poi_lat, poi_lon)

# Create a Folium map centered around the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Create a Marker for the point of interest
poi_marker = folium.Marker(
    [poi_lat, poi_lon],
    icon=folium.DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_to_poi),
    ),
    popup="Closest Point of Interest"
)

# Add the Marker to the map
site_map.add_child(poi_marker)

# Display the map
site_map

In [ ]:
import folium
from math import sin, cos, sqrt, atan2, radians

# Function to calculate distance between two points given their latitude and longitude
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Example coordinates of the launch site (Kennedy Space Center)
launch_site_lat, launch_site_lon = 28.562302, -80.577356

# Example coordinates of the closest city, railway, highway, etc.
poi_lat, poi_lon = 28.533, -81.348

# Calculate the distance between launch site and the point of interest
distance_to_poi = calculate_distance(launch_site_lat, launch_site_lon, poi_lat, poi_lon)

# Create a Folium map centered around the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Create a Marker for the point of interest
poi_marker = folium.Marker(
    [poi_lat, poi_lon],
    icon=folium.DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_to_poi),
    ),
    popup="Closest Point of Interest"
)

# Add the Marker to the map
site_map.add_child(poi_marker)

# Draw a Polyline between launch site and point of interest
line_coordinates = [[launch_site_lat, launch_site_lon], [poi_lat, poi_lon]]
line = folium.PolyLine(locations=line_coordinates, color='blue', weight=2)
site_map.add_child(line)

# Display the map
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
